# Response table for the Scoop task



This code collects the response and response time from each participant's file and save the information in a csv table.

The code first opens each txt file corresponding to the Scoop tasks stored in ***path*** . The ID for each participant is collected from the ***Subject*** line in the txt file, and stored in a variable ***ID***.

Parsing of the file is performed by finding the experimental and control samples for each trial (***NonStroopList1, StroopList1, NonStroopList2, StroopList2, NonStroop3, Strooplist3***) within a ***LogFrame***. Two variables are stored for each sample:

- Response time (***RT***): As found in line ***Slide1.RT***, or 999 if none.
- Response (***Resp***): 1 if the participant answered correrctly (***Response*** = ***Slide1.CRESP***),"video" if the participant touched the screen outside the answer frame (***Response*** = Image#), and 0 otherwise.

Variables are stored as ***trialVar\_S#\_RT***  and ***trialVar\_S#\_Resp***, with *#* being the number of the sample for each trial, and _trialVar_ the variable for each trial as follow:

***NSt1*** for ***NonStroopList1***, ***NSt2*** for ***NonStroopList2***, ***NSt3*** for ***NonStroop3***, ***St1*** for ***StroopList1***, ***St2*** for ***StroopList2***, ***St3*** for ***Strooplist3***.


For example, in the frame corresponding to trial NonStroopList2, sample 1:

	*** LogFrame Start ***
	Procedure: ExpTrial
	TrialType: Nonstroop
	TargetAnimal: Kitten5.bmp
	TargetAudio: catsoundfinal.wav
	Correctresp: Text3
	TriggerValue: 1
	NonStroopList2: 1
	Running: NonStroopList2
	Slide1.OnsetDelay: 141
	Slide1.OnsetTime: 235001
	Slide1.DurationError: 0
	Slide1.RTTime: 237606
	Slide1.ACC: 1
	Slide1.RT: 2605
	Slide1.RESP: 1
	Slide1.CRESP: Text3
	Slide1.OnsetToOnsetTime: 5014
	Response: Text3
	NonStroopList2.Cycle: 1
	NonStroopList2.Sample: 1
	*** LogFrame End ***
    
The value of 2605 is stored in *NSt2\_S1\_RT* and 1 is stored in *NSt2\_S1\_Resp*

Variables are stored in a csv file. There are 73 variables total in the csv file: 6 for each trial plus the participant's ID.

In [5]:
import pandas as pd
import os, sys


In [10]:

path = "/media/ioastorga/Lexar/NIRS_data"
task_files_all = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if ".txt" in file and "Stroop" in file and not "Practice" in file and not "._" in file:
            task_files_all.append(os.path.join(r,file))
print(task_files_all)

['/media/ioastorga/Lexar/NIRS_data/10040_20181215/Pet Store Stroop Task _JM_triggers_B-10040-1.txt', '/media/ioastorga/Lexar/NIRS_data/10046_20190304/Pet Store Stroop Task _JM_triggers-10046-1.txt', '/media/ioastorga/Lexar/NIRS_data/10051_20190425/Pet Store Stroop Task _JM_triggers_StimBlack-10051-1.txt', '/media/ioastorga/Lexar/NIRS_data/10537_20190202/Pet Store Stroop Task _JM_triggers_B-10537-1.txt', '/media/ioastorga/Lexar/NIRS_data/10538_20181208/Pet Store Stroop Task _JM-10538-1.txt', '/media/ioastorga/Lexar/NIRS_data/10560_20190615/Pet Store Stroop Task _JM_triggers_StimBlack-10560-1.txt', '/media/ioastorga/Lexar/NIRS_data/10609_20190223/Pet Store Stroop Task _JM_triggers-10609-1.txt', '/media/ioastorga/Lexar/NIRS_data/10614_20190622/Pet Store Stroop Task _JM_triggers_StimBlack-10614-1.txt', '/media/ioastorga/Lexar/NIRS_data/10616_20190124/Pet Store Stroop Task _JM_triggers_B-10616-1.txt', '/media/ioastorga/Lexar/NIRS_data/10621_20190119/Pet Store Stroop Task _JM_triggers_B-1062

In [11]:
field_to_column = dict([('NonStroopList1','NSt1'), ('StroopList1','St1'),
                   ('NonStroopList2','NSt2'), ('StroopList2','St2'),
                   ('NonStroop3','NSt3'), ('Strooplist3','St3'),]) 


def file_to_row(fname):
    row = {}
    start_parsing_frame = False
    curr_frame = {}
    with open(fname, encoding='utf-16-le') as fl:
        for line in fl.readlines():
            line = line.strip('\n')
            if 'Subject:' in line:
                row['ID'] = line.split(':')[-1].strip()
               # if row['ID'] == '1':
                #    print(fname)
            elif 'LogFrame Start' in line:
                start_parsing_frame = True
            elif 'LogFrame End' in line:
                start_parsing_frame = False
                frame_type = None
                frame_key = None
                for key in curr_frame.keys():
                    for field_key in field_to_column.keys():
                        if key == field_key:
                            frame_type = field_to_column[key] # e.g. Exp1
                            frame_key = key
                if frame_type is None:
                    curr_frame = {}
                else:
                    sample = curr_frame[frame_key] # S: 1-6
                    column_name = frame_type + '_S' + sample
                    if 'Slide1.RT' in curr_frame:
                        row[column_name + '_RT'] = int(curr_frame['Slide1.RT'])
                    else:
                        row[column_name + '_RT'] = 999 
                    if 'Response' in curr_frame and curr_frame['Response'] == curr_frame['Slide1.CRESP']:
                        row[column_name + '_Resp'] = 1
                    elif 'Image' in curr_frame['Response']:
                        row[column_name + '_Resp'] = 'video' # manual input from video?
                    else:
                        row[column_name + '_Resp'] = 0 

                    curr_frame = {}
            elif start_parsing_frame:
                fields = [x.strip() for x in line.split(':')]
                curr_frame[fields[0]] = fields[1]
    return row
            
rows = []

for fname in task_files_all:
    rows.append(file_to_row(fname))

expt_table = pd.DataFrame(rows).set_index('ID')



In [12]:
expt_table.to_csv('table_for_spss_scoop_all.csv')

In [13]:
len(rows)

30